In [13]:
import django_initializer
import pandas as pd
from pandas import json_normalize
import arrow
from backend.pymongo.mongodb import db

In [14]:
def analysis_order(start_time, end_time):
    
    cursor=db.api_campaign.aggregate([
        {
            "$match":{
                "start_at":{
                    "$gte": start_time
                },
                "end_at": {
                    "$lte": end_time
                }
            }
        },
        {
            "$lookup": {
                "from": "api_order",
                "as": "orders", 
                'let': {'id': "$id" },
                "pipeline":[
                    {
                        "$match":{
                            '$expr': { '$eq': ["$$id", "$campaign_id"] },
                            "id":{"$ne":None},
                        }
                    },
                    {"$addFields": { "new_type": "$status"}}
                ]
            },
        },
        {
            "$lookup": {
                "from": "api_pre_order",
                "as": "pre_orders", 
                'let': {'id': "$id" },
                "pipeline":[
                    {
                        "$match":{
                            '$expr': { '$eq': ["$$id", "$campaign_id"] },
                            "id":{"$ne":None},
                            "subtotal": {"$ne" : 0}
                        }
                    },
                    {"$addFields": { "new_type": "cart"}}
                ]
            },
        },
        {"$project":{"_id":0,"data":{"$concatArrays":["$orders","$pre_orders"]}}},
        { "$unwind": "$data" },
        
        { "$group": {
                "_id": {
                    "id": "$data.campaign_id",
                    "status": "$data.new_type"
                },
                "campaign_id": {"$first": "$data.campaign_id"},
                "status": {"$first": "$data.new_type"},
                "qty": {"$sum": 1},
                "total": {"$sum": "$data.total"}
           }
        },
        {
            "$project":{
                "_id":0,

            }
        },
        
        { "$sort" : { "campaign_id" : 1 } }
    ])
    return list(cursor)

In [15]:
start_time = "2022-06-15"
end_time = "2022-06-20"
start_time = arrow.get(start_time).replace(hour=0, minute=0, second=0).datetime
end_time = arrow.get(end_time).replace(hour=23, minute=59, second=59).datetime
report  = analysis_order(start_time, end_time)
display(report)

[{'campaign_id': 494, 'status': 'complete', 'qty': 2, 'total': 322.0},
 {'campaign_id': 494, 'status': 'review', 'qty': 1, 'total': 161.0},
 {'campaign_id': 494, 'status': 'cart', 'qty': 10, 'total': 31597.5},
 {'campaign_id': 495, 'status': 'cart', 'qty': 1, 'total': 15.99},
 {'campaign_id': 496, 'status': 'complete', 'qty': 1, 'total': 127.92},
 {'campaign_id': 496, 'status': 'cart', 'qty': 1, 'total': 31.98},
 {'campaign_id': 497, 'status': 'cart', 'qty': 1, 'total': 15.0},
 {'campaign_id': 497, 'status': 'complete', 'qty': 2, 'total': 25.0},
 {'campaign_id': 499, 'status': 'complete', 'qty': 1, 'total': 8870.0},
 {'campaign_id': 500, 'status': 'complete', 'qty': 2, 'total': 51.0},
 {'campaign_id': 500, 'status': 'cart', 'qty': 2, 'total': 140.0},
 {'campaign_id': 501, 'status': 'cart', 'qty': 2, 'total': 1270.0},
 {'campaign_id': 501, 'status': 'complete', 'qty': 1, 'total': 390.0},
 {'campaign_id': 502, 'status': 'cart', 'qty': 1, 'total': 19.54},
 {'campaign_id': 507, 'status': '

In [80]:
def checkLack(x):
    if len(x) == 1 and x[0] == "paid":
        return "unpaid"
    elif len(x) == 1 and x[0] == "unpaid":
        return "paid"
    else:
        return ""
def insert_0(x):
    cols = ['qty', 'total', 'percentage_of_qty', 'percentage_of_total']
    if x["lack"] == "paid":
        return [[0] + x[col] for col in cols]
    elif x["lack"] == "unpaid":
        return [x[col] + [0] for col in cols]
    else:
        return [x[col] for col in cols]
        
df = json_normalize(report)
df.loc[:,"new_status"] = df["status"].apply(lambda x: 'unpaid' if x in ['review', 'cart'] else 'paid')
df = df.groupby(["campaign_id", "new_status"]).agg({'qty': 'sum', 'total': 'sum'}).reset_index()

df = df.groupby(["campaign_id"]).agg({'new_status': list, 'qty': list, 'total':list}).reset_index()
df.loc[:, "lack"] = df["new_status"].apply(checkLack)


df.loc[:, "percentage_of_qty"] = df["qty"].apply(lambda x: [round(i/sum(x)*100,2) for i in x])
df.loc[:, "percentage_of_total"] = df["total"].apply(lambda x: [round(i/sum(x)*100,2) for i in x])


df["qty"], df["total"], df['percentage_of_qty'], df['percentage_of_total'] = zip(*df.apply(insert_0, axis=1))
df.loc[:, "status"] = df.apply(lambda _: ["paid", "unpaid"], axis=1)

In [81]:
display(df)

,campaign_id,new_status,qty,total,lack,percentage_of_qty,percentage_of_total,status
0,494,"[paid, unpaid]","[2, 11]","[322.0, 31758.5]",,"[15.38, 84.62]","[1.0, 99.0]","[paid, unpaid]"
1,495,[unpaid],"[0, 1]","[0, 15.99]",paid,"[0, 100.0]","[0, 100.0]","[paid, unpaid]"
2,496,"[paid, unpaid]","[1, 1]","[127.92, 31.98]",,"[50.0, 50.0]","[80.0, 20.0]","[paid, unpaid]"
3,497,"[paid, unpaid]","[2, 1]","[25.0, 15.0]",,"[66.67, 33.33]","[62.5, 37.5]","[paid, unpaid]"
4,499,[paid],"[1, 0]","[8870.0, 0]",unpaid,"[100.0, 0]","[100.0, 0]","[paid, unpaid]"
5,500,"[paid, unpaid]","[2, 2]","[51.0, 140.0]",,"[50.0, 50.0]","[26.7, 73.3]","[paid, unpaid]"
6,501,"[paid, unpaid]","[1, 2]","[390.0, 1270.0]",,"[33.33, 66.67]","[23.49, 76.51]","[paid, unpaid]"
7,502,[unpaid],"[0, 1]","[0, 19.54]",paid,"[0, 100.0]","[0, 100.0]","[paid, unpaid]"
8,507,[unpaid],"[0, 2]","[0, 10683.0]",paid,"[0, 100.0]","[0, 100.0]","[paid, unpaid]"
9,509,[unpaid],"[0, 3]","[0, 2700.0]",paid,"[0, 100.0]","[0, 100.0]","[paid, unpaid]"


In [42]:
a = [1,2]
display(sum(a))

3